# Your First RAG Application: Personal Wellness Assistant

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Personal Wellness Assistant**.

Imagine having an AI assistant that can answer your health and wellness questions based on a curated knowledge base - that's exactly what we'll build here. We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading our Wellness Knowledge Base)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Wellness Assistant)
  - 🚧 Activity #1: Enhance Your Wellness Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using a comprehensive Health & Wellness Guide as our sample data. This content covers exercise, nutrition, sleep, stress management, and healthy habits - perfect for building a personal wellness assistant!

In [4]:
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
documents = text_loader.load_documents()
len(documents)
len(documents[0])

16206

In [5]:
print(documents[0][:1000])

The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.

The four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities on 2 or more days per week.

Chapter 2: Exercises for Common Problems

Lower Back Pain Relief
Lower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.

Reco

### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [6]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

21

Let's take a look at some of the documents we've managed to split.

In [8]:
split_documents[0:1]

['The Personal Wellness Guide\nA Comprehensive Resource for Health and Well-being\n\nPART 1: EXERCISE AND MOVEMENT\n\nChapter 1: Understanding Exercise Basics\n\nExercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.\n\nThe four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities on 2 or more days per week.\n\nChapter 2: Exercises for Common Problems\n\nLower Back Pain Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future

In [16]:
split_documents[1:2]

[' Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.\n\nRecommended exercises for lower back pain include:\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [9]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this paper](https://arxiv.org/abs/2205.13147) for an answer to question #1.2!


##### ✅ Answer:
1. Yes this can be modified through the optional `dimensions` parameter according to OpenAI documention: https://platform.openai.com/docs/api-reference/embeddings/create#embeddings_create-dimensions. 
2. This is done via the `Matryoshka Representation Learning` technique. Specifically, traditional embedding models are fixed size vectors, while MRL can train a single embedding so that any prefix (first m dimensions) is itself a useful embedding, which allows the dimension to be customizable depending on the amount of information the corresponding task needs. 

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [ ]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
Async allows the vector database creation task to be executed concurrently across batches, which reduces user-perceived latency.  Specifically, since `abuild_from_list` calls into `async_get_embeddings`, which calls OpenAI's embedding API (I/O-bound), `async` enables the code to run the batch requests in parellel, instead of running each batch request in sequence. For example, with 5 batches, we can save roughly 4 batches worth of latency using `async` vs `sync` (in reality the actual latency improvements by async could be affected by rate limiting, network speed, etc.).

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [19]:
vector_db.search_by_text("What exercises help with lower back pain?", k=3)

[(' Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.\n\nRecommended exercises for lower back pain include:\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs an

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analogous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
- `seed` parameter: We can set the same [seed](https://platform.openai.com/docs/guides/advanced-usage) parameter across API calls and ensure all other parameters like the `prompt` and `temperature` to be the same across requests. This is OpenAI specific. As of Jan 2026, the OpenAI API doc has marked the `seed` parameter as [deprecated](https://platform.openai.com/docs/api-reference/chat/create#chat_create-seed) though. 
- `temperature`: another more genenral parameter originally from the Transformer model to control the randomness/creativity of the reponse. Specifically use a lower value to lower the creativity and make the response more reproducible. 

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [14]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Java"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [15]:
print(response)

Hello! The "best" way to write a loop in Java depends on what you're trying to achieve, but I’m happy to share some general advice to help you write clear and efficient loops.

### Common loop types in Java:

1. **For loop**  
Great when you know exactly how many times you need to iterate.

```java
for (int i = 0; i < 10; i++) {
    System.out.println("Iteration " + i);
}
```

2. **While loop**  
Best when you want to continue looping until a condition changes, and you don’t necessarily know the number of iterations in advance.

```java
int i = 0;
while (i < 10) {
    System.out.println("Iteration " + i);
    i++;
}
```

3. **Do-while loop**  
Similar to while, but guarantees the loop runs at least once.

```java
int i = 0;
do {
    System.out.println("Iteration " + i);
    i++;
} while (i < 10);
```

4. **Enhanced for loop (for-each)**  
Perfect for iterating over arrays or collections without worrying about indexes.

```java
String[] fruits = {"apple", "banana", "cherry"};
for (Strin

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [21]:
RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [22]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [23]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What are some natural remedies for improving sleep quality?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: Based on the provided information, some natural remedies for improving sleep quality include:

1. Cognitive Behavioral Therapy for Insomnia (CBT-I) – a structured therapy to address sleep difficulties (Source 1).
2. Relaxation techniques such as progressive muscle relaxation and meditation to calm the mind and body before sleep (Source 1 and Source 3).
3. Herbal teas like chamomile or valerian root to promote relaxation (Source 1).
4. Magnesium supplements, although it is important to consult a healthcare provider before starting supplements (Source 1).

Additionally, following good sleep hygiene practices enhances natural sleep quality. These include:
- Avoiding caffeine after 2 PM (Sources 1, 2, 3).
- Exercising regularly but not too close to bedtime (Sources 1 and 2).
- Limiting alcohol and heavy meals before bed (Source 1 and 2).
- Creating an optimal sleep environment with a cool bedroom temperature of 65-68°F (18-20°C), darkness via blackout curtains or a sleep mask, qu

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:

One prompt strategy to use for making the LLM to have a more detailed and thoughtful response is to invoke the reasoning capability of the model. For example, we can instruct the model directly in the prompt via chain-of-thought technique such as adding "Think step-by-step" to the prompt. 

In addition, we can also leverage the reasoning control provided via OpenAI when we use the new Response API. Specifically, this can be done via the `reasoning` parameter for the chat model, such as passing `reasoning={"effort": "high"}` to the reponse API for the model to think more carefully and generate a more thoughtful response.

### 🏗️ Activity #1:

Enhance your Personal Wellness Assistant in some way! 

Suggestions are: 

- **PDF Support**: Allow it to ingest wellness PDFs (meal plans, workout guides, medical information sheets)
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for health queries?
- **Metadata Support**: Add metadata like topic categories (exercise, nutrition, sleep) or difficulty levels to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for health content?
- **Multi-Source Ingestion**: Add the capability to ingest content from YouTube health videos, podcasts, or health blogs

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this wellness assistant most useful for your personal health journey.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

### Activity 1.1: Vibe checking current implementation without any enhancements

##### Activity 1.1.1: Retrieval quality tests

Conclusion: After testing a few simple prompts which are covered by the HealthWellness guide, the system passes the vibe check since it can retrieve the relevant content and give a satisfying answer. 

In [ ]:
user_query = "What exercises help with neck and shoulder tension?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")


Response: Exercises that can help with neck and shoulder tension, often caused by desk work and poor posture, include the following:

- Neck Rolls: Slowly roll your head in a circle, 5 times in each direction.
- Shoulder Shrugs: Raise your shoulders toward your ears, hold for 5 seconds, then release. Repeat 10 times.
- Chest Opener: Clasp your hands behind your back, squeeze your shoulder blades together, and lift your arms slightly. Hold this position for 15-30 seconds.
- Chin Tucks: While sitting or standing tall, pull your chin back to create a "double chin." Hold for 5 seconds, then repeat 10 times.

These exercises provide relief by stretching and strengthening the muscles around the neck and shoulders, helping to reduce tension (Source 1). 

As always, please consult with a healthcare professional before beginning any new exercise routine to ensure these exercises are safe and appropriate for your individual health needs.


In [31]:
user_query = "How much water should I drink daily?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")

Response: Based on the information provided, the general recommendation is to drink 8 glasses (64 ounces) of water daily. However, this amount can vary depending on factors such as your activity level, the climate you are in, and your body size (Sources 1 and 2). 

It's also important to watch for signs of dehydration, which include dark yellow urine, fatigue, headaches, dry mouth and skin, and dizziness, so you can adjust your intake accordingly (Sources 1 and 2).

To help stay hydrated, some practical tips include starting your day with a glass of water, carrying a reusable water bottle, setting reminders to drink water, eating water-rich foods like cucumbers, watermelon, and oranges, and drinking a glass of water before each meal (Source 1).

Remember, for personalized advice and if you have specific health conditions, you should consult a healthcare professional.


In [30]:

user_query = "What are some natural remedies for improving sleep quality?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")



Response: Natural remedies for improving sleep quality based on the provided context include:

1. Cognitive Behavioral Therapy for Insomnia (CBT-I) – a therapeutic approach targeting the thoughts and behaviors that cause insomnia (Source 1).

2. Relaxation techniques such as progressive muscle relaxation – systematically tensing and relaxing muscle groups to reduce tension and promote sleep (Sources 1 and 3).

3. Herbal teas like chamomile or valerian root – these plants are traditionally used to promote relaxation (Source 1).

4. Magnesium supplements – which may aid sleep but should be taken after consulting with a healthcare provider (Source 1).

5. Meditation – mindfulness or meditation practices can reduce stress and improve sleep quality (Source 1).

In addition to these remedies, adopting good sleep hygiene practices such as avoiding caffeine after 2 PM, limiting alcohol and heavy meals before bed, keeping a consistent sleep schedule, and creating an optimal sleep environment wi

#### Activity 1.1.2: Vibe check if answer sticks to the knowledge base

Conclusion: the system did mention it doesn't have information from the knowledge base for some reaonsable wellness/fitness related queries. This means we might need to expand our knowledge base to answer those questions

In [33]:
user_query = "What's the best treatment for diabetes?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")



Response: I don't have information about the treatment for diabetes in the provided wellness knowledge base. 

However, for medical advice and treatment of diabetes, it is important to consult healthcare professionals who can provide personalized care and guidance.


In [34]:
user_query = "How does intermitten fasting help with weight loss?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")

Response: I don't have information about intermittent fasting or its effects on weight loss in the provided wellness context. The sources focus on balanced diet principles, meal planning, hydration, and building an exercise routine, but do not mention intermittent fasting specifically.

For personalized advice on intermittent fasting and weight loss, I gently recommend consulting healthcare professionals or registered dietitians who can provide guidance based on your individual health needs.


#### Activity 1.1.3: Vibe check queries that may expose the limitation of the current implementation

Observation:

- Metadata: Without adding metadata like exercise level (beginner, intermediate, pro), then if I ask for beginner level exercise tips, the retrieved chunks do not have high similarity score and cannot guarantee to give answers that are truely beginner friendly. 
- Negation: when asked what exercise are BAD for lower back pain, the 1st retrieved chunk has a very high score but is only talking about exercises that are good for people with lower back pain, not bad for lower back pain. This may require adding more data, as well as potentially updating similarity score metric or re-rank the retrieved chunks based on sentiment/negation. RAG didn't retrieve the relevant text with this example. 
- Chunking strategy: we may also need to improve or tune the chunking strategy so that the RAG system can give a confident answer on morning habits to AVOID (the source text has both morning habits to follow and habits to avoid).



In [42]:
# Test queries that expose limitations
test_cases = [
    "Give me only beginner exercises",
    "What morning habits should I avoid?",
    "What exercises are bad for lower back pain?",
]

for query in test_cases:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    result = rag_pipeline.run_pipeline(query, k=3)
    
    print(f"\nRetrieved chunks:")
    for i, (chunk, score) in enumerate(result['context'], 1):
        print(f"\n[{i}] Score: {score:.3f}")
        print(f"{chunk[:200]}...")  # First 200 chars
    
    print(f"\nResponse: {result['response'][:300]}...")


Query: Give me only beginner exercises

Retrieved chunks:

[1] Score: 0.479
chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.
- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting p...

[2] Score: 0.470
 repeat 10 times.

Chapter 3: Building a Workout Routine

Starting a new exercise routine can feel overwhelming. The key is to start slowly and gradually increase intensity and duration over time.

Be...

[3] Score: 0.415
The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things yo...

Response: Based on the provided context, here are beginner exercises you can try:

1. **Lower Back Pain Relief Exercises:**
   - Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. H

### Activity 1.2: Enhance the RAG system


### Activity 1.2.1 : Support PDF docs

In this enhancement, we will:
- add PDF file loader to our text_utils.py file
- test the PDF file loader with a PDF file added to data/A-STEP-BY-STEP-GUIDE-TO-WELLNESS.pdf
- load both the text and PDF file to create a new vector DB
- test the RAG system with vector DB containing both PDF and text file

In [1]:
# Test PDF support
from aimakerspace.text_utils import PDFFileLoader, CharacterTextSplitter

# Load PDF file
pdf_loader = PDFFileLoader("data/A-STEP-BY-STEP-GUIDE-TO-WELLNESS.pdf")
documents = pdf_loader.load_documents()

# Check what we got
print(f"Number of documents: {len(documents)}")
print(f"Total characters: {len(documents[0])}")
print("\n--- First 1000 characters ---")
print(documents[0][-1000:])



Number of documents: 1
Total characters: 33196

--- First 1000 characters ---
an change your mood and 
attitude. 
Stress • Are you learning to manage stress in ways that 
work for your lifestyle? 
• Do you recognize stress triggers and appreciate 
that you are not your feelings? Feelings are 
fleeting and will pass. 
• Do you welcome and cultivate positive, 
empowering thoughts and emotions? 
• Take a step back when in a stressful situation. 
• Practice deep breathing or other relaxation 
techniques. 
• Try out different coping exercises or strategies when 
not in a stressful situation. When challenges arise, you 
will be better prepared to deal with them. 
• Practice finding positives (a silver lining) in something 
that you feel is negative. Support others in doing this, 
as well. 

22
CREATING A HEALTHIER LIFE: A STEP-BY-STEP GUIDE TO WELLNESS
WAYS TO IMPROVE MY  
EMOTIONAL WELLNESS
WHAT I WILL DO WHAT I NEED
      
      
      
      
      
      
Learn more about the Eight Dimen

In [ ]:
# Split into chunks
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)

print(f"Number of chunks: {len(split_documents)}")
print("\n--- First chunk ---")
print(split_documents[0])

Number of chunks: 42

--- First chunk ---
CREATING A
HEALTHIER LIFE
A STEP-BY-STEP GUIDE TO WELLNESS

CREATING A HEALTHIER LIFE: 
A STEP-BY-STEP GUIDE TO WELLNESS 
The Substance Abuse and Mental Health Services 
Administration’s (SAMHSA) Wellness Initiative envisions 
a future in which people with mental or substance use 
problems pursue health, happiness, recovery, and a full and 
satisfying life in the community.
 
Each individual’s path will be a bit different.
 
Every aspect of wellness can affect a person’s life.
Working toward all of them in one way or another is a great 
goal, because wellness relates directly to the quality of a 
person’s life.  
This guide offers a broad approach for things we can 
do—at our own pace, in our own time, and within our own 
abilities—that can help us feel better and live longer. 

3
CREATING A HEALTHIER LIFE: A STEP-BY-STEP GUIDE TO WELLNESS
WHAT IS WELLNESS?
Wellness is a broad concept. In this guide, we 
attempt to provide a broad, yet specific

In [ ]:
## Add both text and PDF files to the vector DB
from aimakerspace.text_utils import TextFileLoader, PDFFileLoader, CharacterTextSplitter

# Load text file
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
text_documents = text_loader.load_documents()
print(f"Number of text documents: {len(text_documents)}")

# Load PDF file
pdf_loader = PDFFileLoader("data/A-STEP-BY-STEP-GUIDE-TO-WELLNESS.pdf")
pdf_documents = pdf_loader.load_documents()
print(f"Number of PDF documents: {len(pdf_documents)}")

# Combine text and PDF documents

combined_documents = text_documents + pdf_documents
print(f"Total number of combined documents: {len(combined_documents)}")


Number of text documents: 1
Number of PDF documents: 1
Total number of combined documents: 2


In [4]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(combined_documents)
len(split_documents)

63

In [5]:
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

# enable asyncio
import nest_asyncio
nest_asyncio.apply()

# Create a new vector DB
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))



In [6]:
# quick test to see if the vector DB is working
texts = vector_db.search_by_text("How to improve our intellectual wellness?", k=2)
"The Intellectual Wellness Dimension involves many things that keep" in texts[0][0]

True

In [7]:
print(texts[0][0])

STEP GUIDE TO WELLNESS
WAYS TO IMPROVE MY 
PHYSICAL WELLNESS
WHAT I WILL DO WHAT I NEED
      
      
      
      
      
      
9
CREATING A HEALTHIER LIFE: A STEP-BY-STEP GUIDE TO WELLNESS
IMPROVING OUR INTELLECTUAL WELLNESS 
The Intellectual Wellness Dimension involves many things that keep our brains active and 
our intellect expanding. In a broad sense, this dimension can involve looking at different 
perspectives of an issue and taking them into consideration. Through a number of 
activities—from learning about current events to organizing game nights in your home or 
community center—you can broaden your perspective and understand diverse points of 
view.
AREA THINK ABOUT… RESOURCES
Personal Interests • Have you considered teaching a class or leading 
a workshop based on skills, knowledge, or 
experience you have?
• Do you enjoy reading? Might you be interested in 
books, magazines, blogs, Facebook, etc.?
• What would you like to do or learn? Have you 
considered creative arts 

In [8]:
## Let's test the RAG system with the new vector DB

from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()

RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)   

In [9]:
# Verify chunks are retried from both documents

user_query = "What are some tips for better sleep management and better financial wellness?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")

for i, (chunk, score) in enumerate(result['context'], 1):
    print(f"\n[{i}] Score: {score:.3f}")
    print(chunk[:200] + "...")


Response: Based on the provided context, here are some tips for better sleep management and improving financial wellness:

**Tips for Better Sleep Management:**
- Avoid large meals before going to bed.
- Manage your routine to carve out time specifically for rest and sleep.
- Cut back on caffeine intake.
- Limit screen time before bedtime.
- Explore resources and tips from reputable sources like HealthyPeople.gov, Mayo Clinic, and the Centers for Disease Control and Prevention for more guidance on sleep (Source 3).
- Consider using tools to help adjust your sleep schedule and meal planning around sleep (Source 1).

**Tips for Improving Financial Wellness:**
- Reflect on how financial wellness affects your overall life and wellness.
- Assess whether your current job allows you to meet your financial obligations and provides resources to enjoy activities you like.
- Consider if you are working in a field you are passionate about or do well, or if you are looking to change to something di

### Activity 1.2.2 : Support additional sources: Youtube video, Blogs

In this enhancement, we will:
- support additional sources beyond offline txt and PDF files, this includes Youtube videos, blogs.
- Created `YoutubeVideoLoader` and `BlogPostLoader` in text_utils.py to process youtube URL and blog URL respectively.
- Added 1 youtube video and 1 blog post to our source documents and updated vector DB
- Showed the RAG will retrieve from across our sources to answer a query like "How to reduce anxiety?".

In [19]:
## Support Youtube video source
from aimakerspace.text_utils import YoutubeVideoLoader, CharacterTextSplitter

yt_loader = YoutubeVideoLoader("https://www.youtube.com/watch?v=rKQLBiylsn8")
yt_documents = yt_loader.load_documents()

print(yt_documents[0][:1000])  # Preview first 1000 chars

[Music] tedx Memphis it is an honor to be here and also pretty timely if you ask me see I'm fresh on the other side of my quarter life crisis now to be clear the crisis is still here I just turned 26 so your girl aged out I'm on the other side of that quarter life side and if you're not familiar a quarter life crisis is basically like a midlife crisis just 25 years early so you're still asking the same questions of Who am I am I using my time well what are the things that I really want to do in terms of family career health drinking water saying hydrated all of those important things that we have to pay attention to and for Millennials this is actually a pretty big phenomenon according to the Center for Disease Control one in three Millennials have a diagnosed anxiety disorder and anxiety is one of the key elements of any kind of crisis it's proven math I promise and for us it's not just about figuring those things out but we're doing so in a world that really human history has never s

In [11]:
## Support Blog post URL source
from aimakerspace.text_utils import BlogPostLoader

blog_loader = BlogPostLoader("https://www.health.harvard.edu/blog/5-timeless-habits-for-better-health-202509243106")
blog_documents = blog_loader.load_documents()

print(blog_documents[0][:1000])  # Preview first 1000 chars


5 timeless habits for better health
Adopting a healthier way of life moves us beyond quick fixes and fads.
- Reviewed by Wynne Armand, MD, Contributor
A veritable avalanche of influencers, news headlines, and well-meaning friends and family offer conflicting advice on wellness. If you’re overwhelmed by fads and hype around wellness, read on. Dr. Wynne Armand, a primary care physician at Harvard-affiliated Mass General Brigham health care in Boston, recommends five simple habits to boost health and wellness in everyday life.
Five wellness habits for life
Quick fixes and fads may do little for our health, and some even prove harmful. Try to work healthy habits into your daily routines. If you’re a parent, sharing and modeling them can help children and teens embrace wellness, too.
Bring mindfulness into your day
Mindfulness and meditation are overlapping techniques proven to ease stress, improve sleep, and enhance concentration. “Anxiety and stress are very common issues in my practice,”

In [12]:
## Now let's combine the previous documents and the new ones

combined_documents = text_documents + pdf_documents + yt_documents + blog_documents

print(f"Total number of combined documents: {len(combined_documents)}")



Total number of combined documents: 4


In [13]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(combined_documents)
len(split_documents)

93

In [14]:
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

# enable asyncio
import nest_asyncio
nest_asyncio.apply()

# Create a new vector DB
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))


In [16]:
# quick test to see if the vector DB is working
texts = vector_db.search_by_text("box breathing technique", k=3)

for text in texts:
    print(f"Text: {text[0][:100]}...")  # Preview first 100 chars
    print(f"Relevance Score: {text[1]}\n")

"One exercise that works equally well for adults and younger people is box breathing" in texts[0][0]

Text: ion knits together body and mind by focusing on sensations, such as breathing, images, or a repeated...
Relevance Score: 0.4249115588179947

Text: chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.
- Pelvic Tilts: L...
Relevance Score: 0.37522004680421644

Text: gement involves both immediate relief techniques and long-term lifestyle changes.

Immediate stress ...
Relevance Score: 0.3366226119333453



True

In [17]:
## Let's test the RAG system with the new vector DB

from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()

RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)   



In [18]:
# Verify chunks are retried from blog and youtube

user_query = "How can I reduce anxiety?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=4,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")

for i, (chunk, score) in enumerate(result['context'], 1):
    print(f"\n[{i}] Score: {score:.3f}")
    print(chunk[:200] + "...")


Response: To reduce anxiety, you can incorporate mindfulness and meditation techniques into your daily routine as these are proven to ease stress and enhance concentration (Source 1). Mindfulness involves anchoring yourself in the present moment, such as appreciating nature during a walk or using mindfulness apps like Calm and Headspace. This practice can calm anxiety and amplify positive feelings.

Meditation, including exercises like box breathing, helps knit together body and mind by focusing on sensations such as breathing or repeating a word or phrase (Source 1). Box breathing is a simple technique that adults and younger people can use.

For immediate anxiety relief, practice deep breathing techniques such as inhaling for 4 counts, holding for 4, and exhaling for 4 (Sources 3, 4). Other immediate relief techniques include progressive muscle relaxation (tensing and releasing muscle groups), grounding techniques (naming things you see, hear, feel, smell, and taste), taking short wa

#### 1.2.3: Support metadata
In this assignment, I have implemented source metadata, so that when answering user queries, our system can also point to exactly which source the answer comes from. 

In the code below, we have added metadata info that includes the source (e.g., YT video link, PDF/TXT file name etc.). When chunking each source data file through CharacterTextSplitter, we will add the source information metadata to each chunk. 

Then we updated VectorDatabase class to add a `abuild_from_documents` method, which for each chunk of text, it will add both its embedding vector and metadata into the database.

Finally we updated our RAG system prompt and the RAG pipeline, such that when answering user query, it will cite the specific source (e.g., URL, file name) for the given answer, so that users will know where does the answer come from and dive deeper into the source if needed. This is much more helpful than giving a generic source citing like "Source 1", "Source 2", etc.

Note: In a production system with a lot more data sources, we can add more metadata such as categories (e.g., physical health, mental health, etc) to each data chunk. The metadata can be either added manually, or use a classification model. During the retrival phase, we can infer from user query which metadata tag to filter the chunks to retrieve from, and then perform the actual retrival so that the results could be more relevant to user query, also the retrival step can finish faster due to metadata/category filtering.  

In [1]:
## Support Youtube video source
from aimakerspace.text_utils import YoutubeVideoLoader, BlogPostLoader, TextFileLoader, PDFFileLoader, Document

yt_loader = YoutubeVideoLoader("https://www.youtube.com/watch?v=rKQLBiylsn8")
yt_texts = yt_loader.load_documents()
yt_documents = [Document(text=t, metadata={"source": "https://www.youtube.com/watch?v=rKQLBiylsn8", "type": "video"}) for t in yt_texts]

print(yt_documents[0].metadata)  # Preview metadata

blog_loader = BlogPostLoader("https://www.health.harvard.edu/blog/5-timeless-habits-for-better-health-202509243106")
blog_texts = blog_loader.load_documents()
blog_documents = [Document(text=t, metadata={"source": "https://www.health.harvard.edu/blog/5-timeless-habits-for-better-health-202509243106", "type": "blog"}) for t in blog_texts]
print(blog_documents[0].metadata)

# Load text file
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
text_texts = text_loader.load_documents()
text_documents = [Document(text=t, metadata={"source": "data/HealthWellnessGuide.txt", "type": "text"}) for t in text_texts]
print(text_documents[0].metadata)

# Load PDF file
pdf_loader = PDFFileLoader("data/A-STEP-BY-STEP-GUIDE-TO-WELLNESS.pdf")
pdf_texts    = pdf_loader.load_documents()
pdf_documents = [Document(text=t, metadata={"source": "data/A-STEP-BY-STEP-GUIDE-TO-WELLNESS.pdf", "type": "pdf"}) for t in pdf_texts]
print(pdf_documents[0].metadata)


{'source': 'https://www.youtube.com/watch?v=rKQLBiylsn8', 'type': 'video'}
{'source': 'https://www.health.harvard.edu/blog/5-timeless-habits-for-better-health-202509243106', 'type': 'blog'}
{'source': 'data/HealthWellnessGuide.txt', 'type': 'text'}
{'source': 'data/A-STEP-BY-STEP-GUIDE-TO-WELLNESS.pdf', 'type': 'pdf'}


In [2]:
from aimakerspace.text_utils import CharacterTextSplitter

combined_documents = text_documents + pdf_documents + yt_documents + blog_documents

print(f"Total number of combined documents: {len(combined_documents)}")


text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_documents(combined_documents)

print(f"Number of chunks: {len(split_documents)}")
print("\n--- First chunk Metadata ---")
print(split_documents[0].metadata)

Total number of combined documents: 4
Number of chunks: 93

--- First chunk Metadata ---
{'source': 'data/HealthWellnessGuide.txt', 'type': 'text', 'chunk_index': 0}


In [3]:
from aimakerspace.vectordatabase import VectorDatabase

# Build vector DB
vector_db = VectorDatabase()
vector_db = await vector_db.abuild_from_documents(split_documents)

In [10]:
# Search returns metadata
results = vector_db.search_by_text("how to practice with lower back pain", k=3)
for text, score, metadata in results:
    print(f"Source: {metadata.get('source')} | Chunk: {metadata.get('chunk_index')} | Type: {metadata.get('type')} | Score: {score:.3f}")

Source: data/HealthWellnessGuide.txt | Chunk: 1 | Type: text | Score: 0.665
Source: data/HealthWellnessGuide.txt | Chunk: 2 | Type: text | Score: 0.457
Source: data/HealthWellnessGuide.txt | Chunk: 0 | Type: text | Score: 0.398


In [ ]:
## Let's update the RAG system to instruct the LLM to cite the specific source when answering user query

from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()

RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite the source (e.g., [Source 1 - wellness.pdf]) when using information from the context
- When citing sources, always include the full source name, not just the number (e.g., say "according to health.harvard.edu" or "from HealthWellnessGuide.txt", NOT just "Source 1")
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        sources = []
        
        for i, (context, score, metadata) in enumerate(context_list, 1):
            source = metadata.get('source')
            sources.append(source)
            context_prompt += f"[Source {i} - {source}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "sources": sources,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)   



In [ ]:
# Verify that when answering user queries,the LLM will cite the specific source name

user_query = "How can I reduce anxiety?"

result = rag_pipeline.run_pipeline(
    user_query,
    k=4,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")

for i, (chunk, score, metadata) in enumerate(result['context'], 1):
    print(f"\n[{i}] Score: {score:.3f}")
    print(chunk[:200] + "...")

Response: To reduce anxiety, several techniques and lifestyle approaches proven to ease stress and calm the mind are recommended:

1. Mindfulness and Meditation:  
   - Practicing mindfulness helps anchor you in the present moment, reducing worrying about the past or future which often drives anxiety. Mindfulness can be as simple as appreciating nature during a walk or using apps like Calm or Headspace.  
   - Meditation focuses on bodily sensations such as breathing or repeating a phrase, helping knit together body and mind. A specific meditation technique mentioned is box breathing (inhale, hold, exhale, hold for equal counts) which is helpful for calming anxiety.  
(According to https://www.health.harvard.edu/blog/5-timeless-habits-for-better-health-202509243106)

2. Deep Breathing and Relaxation Techniques:  
   - Deep breathing exercises, such as inhaling for 4 counts, holding for 4, and exhaling for 4, provide immediate relief from anxiety.  
   - Progressive muscle relaxation, w

### 1.2.4: Embedding and similarity search metrics

**Embedding Model Choice:**
We used OpenAI's `text-embedding-3-small` model, which is a strong general-purpose embedding model. For example, during vibe checking, I have asked about a medical acronym, and the RAG is able to retrieve documents that has information related to the full form of the acronym. 

For this wellness assistant use case, domain-specific embedding models could potentially improve retrieval quality, but OpenAI's embeddings are robust enough for our scale. This would be worth exploring in a production system with more niche and domain-specific applications.

**Distance Metric Choice:**
We used cosine similarity as our distance metric, which should be standard choice for on embedding models like OpenAI. Also given our application is text-only retrieval so far, cosine similarity works really well.

Alternative metrics such as Euclidean and Manhattan also exists, which may work better in some other application/domain scenarios like image embeddings. 

**Conclusion:** No changes made - cosine similarity is already the right choice for OpenAI embeddings.